<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_convnets/class_03/9%20-%20C2_W3_Lab_1_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Aprendizagem por transferência(_Transfer Learning_)

Neste laboratório, você verá como usar um modelo pré-treinado para obter bons resultados mesmo com um conjunto de dados de treinamento pequeno.

Isso é chamado de _aprendizagem por transferência_ e você faz isso aproveitando as camadas treinadas de um modelo existente e adicionando suas próprias camadas para se adequar a sua aplicação.

Por exemplo, você pode:

1. obter apenas as camadas de convolução de um modelo
2. anexar algumas camadas densas a ele
3. treinar apenas a rede densa
4. avaliar os resultados

Com isso, você economizará tempo na criação do aplicativo porque, basicamente, pulará semanas de treinamento de redes muito profundas. Você usará apenas os recursos que ela aprendeu e os ajustará para o seu conjunto de dados. Vamos ver como isso é feito nas próximas seções.

## Configurar o modelo pré-treinado

Você precisará preparar o modelo pré-treinado e configurar as camadas necessárias. Para este exercício, você usará as camadas de convolução da arquitetura [InceptionV3](https://arxiv.org/abs/1512.00567) como modelo de base. Para fazer isso, você precisa:

1. Definir a forma de entrada de acordo com sua aplicação. Nesse caso, defina-a como `150x150x3`, como você fez nos últimos laboratórios.

2. Selecionar e congelar as camadas de convolução para aproveitar os recursos que já foram aprendidos.

3. Adicione camadas densas que você treinará.

Vamos ver como fazer isso nas próximas células.

Primeiro, ao preparar a entrada para o modelo, você deseja obter os pesos pré-treinados do modelo `InceptionV3` e remover a camada totalmente conectada no final, pois ela será substituída posteriormente.

Você também especificará a forma de entrada que o modelo aceitará. Por fim, você deseja congelar os pesos dessas camadas porque elas já foram treinadas.

In [ ]:
# Faça o download dos pesos pré-treinados.
# Sem topo(no top) significa que ele exclui a camada totalmente conectada que usa para classificação.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# Defina o arquivo de pesos que você baixou em uma variável
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Inicialize o modelo básico.
# Defina a forma de entrada e remova as camadas densas.
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

# Carregue os pesos pré-treinados que você baixou.
pre_trained_model.load_weights(local_weights_file)

# Congelar os pesos das camadas.
for layer in pre_trained_model.layers:
  layer.trainable = False

Você pode ver o resumo do modelo abaixo.

Você pode ver que se trata de uma rede muito profunda. Você pode então selecionar até qual ponto da rede deseja usar.

Você usará até `mixed7` como modelo de base.

Isso se deve ao fato de que a última camada original pode ser muito especializada no que aprendeu e, portanto, pode não se adaptar bem a sua aplicação.

O `mixed7`, por outro lado, será mais generalizado e você pode começar com ele para o seu problema.

Após o exercício, sinta-se à vontade para modificar e usar outras camadas para ver os resultados que você obtém.

In [ ]:
pre_trained_model.summary()


In [ ]:
# Escolha `mixed7` como a última camada de seu modelo básico
last_layer = pre_trained_model.get_layer('mixed7')
print('formato de saída da última camada: ', last_layer.output_shape)
last_output = last_layer.output

## Adicione camadas densas ao seu modelo

Em seguida, você adicionará camadas densas ao seu modelo.

Essas serão as camadas que você treinará e terão a tarefa de reconhecer cães e gatos.

Você também adicionará uma camada [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) para regularizar a saída e evitar o ajuste excessivo.

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

# Achatar a camada de saída para uma dimensão
x = layers.Flatten()(last_output)
# Adicione uma camada totalmente conectada com 1.024 unidades ocultas e ativação ReLU
x = layers.Dense(1024, activation='relu')(x)
# Adicionar uma taxa de dropout de 0,2
x = layers.Dropout(0.2)(x)                  
# Adicionar uma camada sigmoide final para classificação
x = layers.Dense  (1, activation='sigmoid')(x)           

# Anexar a rede densa ao modelo básico
model = Model(pre_trained_model.input, x) 

# Imprima o resumo do modelo. Veja sua rede densa conectada no final.
model.summary()

In [ ]:
# Definir os parâmetros de treinamento
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

## Preparar o conjunto de dados

Agora você vai preparar o conjunto de dados.

Esse código é basicamente o mesmo que você usou no laboratório de aumento de dados.

In [ ]:
# Baixar o conjunto de dados
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

In [ ]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Extrair o arquivo
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Definir nossos diretórios e arquivos de exemplo
base_dir = 'tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

# Diretório com fotos de treinamento de gatos
train_cats_dir = os.path.join(train_dir, 'cats') 

# Diretório com fotos de treinamento de cães
train_dogs_dir = os.path.join(train_dir, 'dogs') 

# Diretório com imagens de gatos para validação
validation_cats_dir = os.path.join(validation_dir, 'cats') 

# Diretório com fotos de cães para validação
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Adicione nossos parâmetros de aumento de dados ao ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Observe que os dados de validação não devem ser aumentados!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Fluxo de imagens de treinamento em lotes de 20 usando o gerador train_datagen
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Fluxo de imagens de validação em lotes de 20 usando o gerador test_datagen
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

## Treinar o modelo

Com isso, agora você pode treinar o modelo. Você fará 20 épocas e plotará os resultados depois.

In [ ]:
# Treine o modelo.
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)

## Avaliar os resultados

Você usará o mesmo código para plotar os resultados.

Como você pode ver, a precisão da validação também está tendendo para cima à medida que a precisão do treinamento melhora.

Esse é um bom sinal de que seu modelo não está mais se ajustando demais!

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Acurácia de Treino')
plt.plot(epochs, val_acc, 'b', label='Acurácia de Validação')
plt.title('Acurácia de Treino e Validação')
plt.legend(loc=0)
plt.figure()


plt.show()